# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846721373268                   -0.70    4.5
  2   -7.852300546703       -2.25       -1.54    1.0
  3   -7.852646215197       -3.46       -2.52    3.2
  4   -7.852646678059       -6.33       -3.41    2.2
  5   -7.852646686205       -8.09       -4.61    1.8
  6   -7.852646686728       -9.28       -5.83    3.0
  7   -7.852646686730      -11.82       -7.06    2.5
  8   -7.852646686730      -13.91       -7.54    2.8
  9   -7.852646686730   +  -14.75       -8.49    2.5
 10   -7.852646686730      -14.75      -10.11    2.5
 11   -7.852646686730   +  -14.75      -10.73    3.8
 12   -7.852646686730   +  -14.75       -9.85    2.0
 13   -7.852646686730      -14.75       -8.97    3.2
 14   -7.852646686730   +  -15.05       -8.27    4.2
 15   -7.852646686730      -14.57       -8.28    4.2
 16   -7.852646686730   +  -14.21       -8.47    3.2
 17   -7.852646686730      -14.35       -8.47 

## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846775837069                   -0.70           4.8
  2   -7.852525389414       -2.24       -1.64   0.80    2.2
  3   -7.852635613211       -3.96       -2.74   0.80    1.0
  4   -7.852646583404       -4.96       -3.23   0.80    2.2
  5   -7.852646681104       -7.01       -4.10   0.80    1.2
  6   -7.852646686310       -8.28       -4.90   0.80    1.5
  7   -7.852646686720       -9.39       -5.82   0.80    2.0
  8   -7.852646686730      -11.01       -6.60   0.80    2.0
  9   -7.852646686730      -12.70       -7.29   0.80    2.2
 10   -7.852646686730      -14.75       -7.96   0.80    1.5
 11   -7.852646686730      -15.05       -8.47   0.80    2.2
 12   -7.852646686730      -15.05       -8.80   0.80    2.0
 13   -7.852646686730   +  -14.75       -9.56   0.80    1.5
 14   -7.852646686730      -14.75      -10.26   0.80    1.8
 15   -7.852646686730   +  -14.75      -

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.363936e+01     3.527092e+00
 * time: 0.4305901527404785
     1     1.555842e+00     1.805304e+00
 * time: 0.6248800754547119
     2    -1.481519e+00     2.046385e+00
 * time: 0.7442581653594971
     3    -3.653171e+00     1.775244e+00
 * time: 0.7779140472412109
     4    -5.008097e+00     1.472117e+00
 * time: 0.8118040561676025
     5    -6.716980e+00     9.121510e-01
 * time: 0.845440149307251
     6    -6.754167e+00     1.543065e+00
 * time: 0.8685891628265381
     7    -7.497812e+00     1.326442e+00
 * time: 0.8919780254364014
     8    -7.543500e+00     2.410042e+00
 * time: 0.9151041507720947
     9    -7.636287e+00     6.975646e-01
 * time: 0.9487631320953369
    10    -7.681552e+00     1.562454e+00
 * time: 0.9826879501342773
    11    -7.778348e+00     7.690697e-01
 * time: 1.0062580108642578
    12    -7.806236e+00     1.902862e-01
 * time: 1.0501079559326172
    13    -7.826908e+00     1.818753e-01
 * time: 1.07329106330

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846789761058                   -0.70    4.5
  2   -7.852297574243       -2.26       -1.53    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686425                   -2.54
  2   -7.852646686730       -9.52       -5.68
  3   -7.852646686730      -14.45      -10.76
  4   -7.852646686730   +  -14.45      -15.31


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 4.634998098597339e-13
|ρ_newton - ρ_scfv| = 2.244389012718364e-12
|ρ_newton - ρ_dm|   = 1.9165828972484852e-10
